[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO for Football Player Tracking and Heatmap Generation**

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## Objective
This notebook provides a complete workflow for building a football analytics tool using computer vision. We will fine-tune a **YOLO (You Only Look Once)** model to detect and track players on a football pitch. Using this model, we will generate advanced analytics like **player trajectories** and **positional heatmaps**.

## Key Features
* **Data Preparation**: Convert annotations from COCO format to the YOLO format required for training.
* **Model Training**: Fine-tune a pre-trained YOLO model on a custom football dataset.
* **Player Tracking**: Apply the trained model to track individual players across video frames.
* **Trajectory Visualization**: Draw the paths of players to analyze movement patterns.
* **Heatmap Generation**: Create heatmaps to visualize team positioning and field control.

## Libraries & Prerequisites
* **Core Libraries**: `ultralytics`, `opencv-python`, `matplotlib`, `numpy`.
* **Environment**: A Python environment with GPU support (like Google Colab) is highly recommended for efficient model training.
* **Dataset**: You'll need a custom dataset of football images with annotations in COCO format (`.json` file) and the corresponding video files for inference.

## **Environment Setup**

First, let's set up our environment by cloning the utility repository, which contains the helper function to convert our dataset annotations from COCO to YOLO format.

In [ ]:
# Clone the utility repository to access the required functions
!git clone https://github.com/yashsuman15/yolo_finetune_utils.git

## **Dataset Preparation**

This step prepares our dataset for training. The key function we'll use is `coco_to_yolo_converter`, which transforms annotations from the common COCO format into the specific YOLO `.txt` format required by the `ultralytics` library.

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.bbox_converter import coco_to_yolo_converter
# Convert COCO annotations to YOLO format
# Ensure the paths are correct and the dataset_annotation.json is in COCO format
# The images_dir should contain the images dataset
# The json_path should point to the COCO annotations file
result = coco_to_yolo_converter(
            json_path='./dataset_annotation.json',
            images_dir='./dataset',
            output_dir='yolo_format',
            use_split=False
            )

## **Training the YOLO Model**

With our dataset properly formatted, we can now fine-tune the YOLO model. We'll start by checking the `ultralytics` installation. Then, we define the path to our dataset's configuration file (`dataset.yaml`) and launch the training process using a pre-trained model. This leverages **transfer learning** to adapt the model to our specific task.

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [ ]:
!pwd

In [ ]:
location = !pwd
dataset_path = f"{location[0]}/yolo_format"
print(f"Dataset path: {dataset_path}")

In [ ]:
!yolo task=detect mode=train data={dataset_path}/dataset.yaml model="yolo11x.pt" epochs=200 imgsz=640 batch=20

## **Tracking Player on the Field**

Now that our model is trained, we can use it for tracking players in videos. We'll load our custom-trained weights and apply them to new video files.

To quickly verify that our model's tracking is working, let's test it on a single video frame. We will load our new weights, run the tracker, and use `matplotlib` to visualize the bounding boxes and track IDs.

In [ ]:
model = YOLO('./runs/detect/train2/weights/last.pt')

In [ ]:
results = model.track(source="./Video/3.mp4", persist=True, stream=True)

# Find frame #30
for frame_idx, res in enumerate(results):
    if frame_idx < 30:
        continue

    # Grab img, boxes, track-IDs and class-IDs
    frame_rgb = cv2.cvtColor(res.orig_img, cv2.COLOR_BGR2RGB)
    boxes     = res.boxes.xyxy.cpu().numpy()    # (N,4)
    track_ids = res.boxes.id.cpu().numpy()      # (N,)
    class_ids = res.boxes.cls.cpu().numpy().astype(int)  # (N,)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(frame_rgb)
    ax.axis('off')

    team_colors = {0: 'red', 4: 'blue'}  # map your relevant class IDs → colors

    # Now zip over the three arrays, using a different name than `cls`
    for (x1, y1, x2, y2), tid, cid in zip(boxes, track_ids, class_ids):
        # print out for debug
        print(f"Player ID: {tid}, Class: {cid}")

        # only draw if the class is in your team mapping
        if cid in team_colors:
            w, h = x2 - x1, y2 - y1
            color = team_colors[cid]

            # draw box
            rect = plt.Rectangle(
                (x1, y1), w, h,
                linewidth=2, edgecolor=color, facecolor='none'
            )
            ax.add_patch(rect)

            # label with track ID
            ax.text(
                x1, y1 - 6, f"Player {int(tid)}",
                color='white', fontsize=7,
                bbox=dict(facecolor=color, alpha=0.5, pad=1, linewidth=0)
            )

    plt.show()
    break


In [ ]:
# Input and output paths
video_path = "./Video/2.mp4"
output_path = "./Video/2_tracked-2.mp4"

# Open video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Team color mapping
team_colors = {0: (0, 0, 255), 4: (255, 0, 0)}  # Red for class 0, Blue for class 4 (BGR)

# Run tracking on the video as a stream
results = model.track(source=video_path, persist=True, stream=True, )

# Process frame by frame
for res in results:
    frame = res.orig_img.copy()  # BGR format for saving with OpenCV

    if res.boxes.id is None:
        out.write(frame)
        continue

    boxes     = res.boxes.xyxy.cpu().numpy()
    track_ids = res.boxes.id.cpu().numpy().astype(int)
    class_ids = res.boxes.cls.cpu().numpy().astype(int)

    for (x1, y1, x2, y2), tid, cid in zip(boxes, track_ids, class_ids):
        if cid in team_colors:
            color = team_colors[cid]
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Draw label
            label = f"Player {tid}"
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(frame)

# Release everything
cap.release()
out.release()
print("✅ Tracking video saved at:", output_path)


## **Tracking the Trajectory of the Player**

Beyond simple tracking, we can visualize player movement by plotting their trajectories. We'll store the center point of each player's bounding box over time and draw lines to create a visual trail showing each player's path.

In [ ]:
from collections import defaultdict

# Input and output video paths
video_path = "./Video/4.mp4"
output_path = "./Video/4_output_with_trajectory.mp4"

# Open video and get properties
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Team color mapping (BGR)
team_colors = {0: (0, 0, 255), 4: (255, 0, 0)}  # Red for class 0, Blue for class 4

# Dictionary to store trajectory points
trajectories = defaultdict(list)

# Perform tracking
results = model.track(source=video_path, persist=True, stream=True)

for res in results:
    frame = res.orig_img.copy()

    if res.boxes.id is None:
        out.write(frame)
        continue

    boxes     = res.boxes.xyxy.cpu().numpy()
    track_ids = res.boxes.id.cpu().numpy().astype(int)
    class_ids = res.boxes.cls.cpu().numpy().astype(int)

    for (x1, y1, x2, y2), tid, cid in zip(boxes, track_ids, class_ids):
        if cid in team_colors:
            color = team_colors[cid]
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

            # Update trajectory for this track_id
            trajectories[tid].append((cx, cy))

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"Player {tid}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Draw trajectory line (at least 2 points needed)
            if len(trajectories[tid]) >= 2:
                for j in range(1, len(trajectories[tid])):
                    pt1 = trajectories[tid][j - 1]
                    pt2 = trajectories[tid][j]
                    cv2.line(frame, pt1, pt2, color, 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Video with trajectories saved to:", output_path)


## **Heatmap of the Players on the ground**

To get a high-level view of player positioning, we can generate a heatmap. This visualization aggregates player locations over the entire video. We create an accumulator for each team and increment the values where players are detected. Finally, we apply a colormap and overlay it on a video frame to see which areas were most occupied.

In [ ]:
import numpy as np

video_path = "./Video/4.mp4"

classes_of_interest = [0, 4]  # your team classes

# Open video to get shape & frame count
cap = cv2.VideoCapture(video_path)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

# 2. Initialize accumulators: one 2D float array per class
heatmaps = {cls: np.zeros((height, width), dtype=np.float32) 
            for cls in classes_of_interest}

# 3. Run tracking (or detection) over entire video
results = model.track(source=video_path, persist=True, stream=True)

for res in results:
    if res.boxes.id is None:
        continue

    boxes     = res.boxes.xyxy.cpu().numpy().astype(int)
    class_ids = res.boxes.cls.cpu().numpy().astype(int)

    for (x1, y1, x2, y2), cid in zip(boxes, class_ids):
        if cid not in classes_of_interest:
            continue

        # Option A: accumulate box area
        heatmaps[cid][y1:y2, x1:x2] += 1

        # —OR— Option B: accumulate only the center point
        # cx, cy = (x1 + x2)//2, (y1 + y2)//2
        # heatmaps[cid][cy, cx] += 1

# 4. Normalize & colorize each heatmap
colored_maps = {}
for cid, hm in heatmaps.items():
    # normalize to 0–255
    hm_norm = cv2.normalize(hm, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    # apply JET colormap
    colored = cv2.applyColorMap(hm_norm, cv2.COLORMAP_JET)
    colored_maps[cid] = colored  # BGR image

# 5. Overlay on a sample frame (e.g., the first frame)
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)

ret, base = cap.read()
cap.release()
if not ret:
    raise RuntimeError("Failed to read sample frame.")

overlay = base.copy()
alpha = 0.5  # transparency

for cid, cmap in colored_maps.items():
    # blend heatmap with the base frame
    cv2.addWeighted(cmap, alpha, overlay, 1 - alpha, 0, overlay)

# 6. Save or display
cv2.imwrite("class_heatmaps_overlay.png", overlay)
print("Saved overlay image with heatmaps: per_class_heatmaps_overlay.png")